In [11]:
import pandas as pd

In [12]:
df = pd.read_csv(r"../data/Trips_by_Distance.csv")

In [13]:
df2 = pd.read_csv(r"../data/Trips_Full Data.csv")

In [14]:
[(df.columns[i] or None,df2.columns[i] or None) for i in range(max(len(df.columns),len(df2.columns)))]

[('Level', 'Month of Date'),
 ('Date', 'Week of Date'),
 ('State FIPS', 'Year of Date'),
 ('State Postal Code', 'Level'),
 ('County FIPS', 'Date'),
 ('County Name', 'Week Ending Date'),
 ('Population Staying at Home', 'Trips <1 Mile'),
 ('Population Not Staying at Home', 'People Not Staying at Home'),
 ('Number of Trips', 'Population Staying at Home'),
 ('Number of Trips <1', 'Trips'),
 ('Number of Trips 1-3', 'Trips 1-25 Miles'),
 ('Number of Trips 3-5', 'Trips 1-3 Miles'),
 ('Number of Trips 5-10', 'Trips 10-25 Miles'),
 ('Number of Trips 10-25', 'Trips 100-250 Miles'),
 ('Number of Trips 25-50', 'Trips 100+ Miles'),
 ('Number of Trips 50-100', 'Trips 25-100 Miles'),
 ('Number of Trips 100-250', 'Trips 25-50 Miles'),
 ('Number of Trips 250-500', 'Trips 250-500 Miles'),
 ('Number of Trips >=500', 'Trips 3-5 Miles'),
 ('Row ID', 'Trips 5-10 Miles'),
 ('Week', 'Trips 50-100 Miles'),
 ('Month', 'Trips 500+ Miles')]

In [15]:
[(df.columns[i].replace('Number of ','') or None,df2.columns[i].replace(' of Date','').replace(' Miles','') or None) for i in range(max(len(df.columns),len(df2.columns)))]

[('Level', 'Month'),
 ('Date', 'Week'),
 ('State FIPS', 'Year'),
 ('State Postal Code', 'Level'),
 ('County FIPS', 'Date'),
 ('County Name', 'Week Ending Date'),
 ('Population Staying at Home', 'Trips <1 Mile'),
 ('Population Not Staying at Home', 'People Not Staying at Home'),
 ('Trips', 'Population Staying at Home'),
 ('Trips <1', 'Trips'),
 ('Trips 1-3', 'Trips 1-25'),
 ('Trips 3-5', 'Trips 1-3'),
 ('Trips 5-10', 'Trips 10-25'),
 ('Trips 10-25', 'Trips 100-250'),
 ('Trips 25-50', 'Trips 100+'),
 ('Trips 50-100', 'Trips 25-100'),
 ('Trips 100-250', 'Trips 25-50'),
 ('Trips 250-500', 'Trips 250-500'),
 ('Trips >=500', 'Trips 3-5'),
 ('Row ID', 'Trips 5-10'),
 ('Week', 'Trips 50-100'),
 ('Month', 'Trips 500+')]

In [21]:
import pandas as pd
from rapidfuzz import process, fuzz



# Preprocess the column names if needed
df_cols = [col.replace('Number of ', '') for col in df.columns]
df2_cols = [col.replace(' of Date', '').replace(' Miles', '').replace('Mile', '') for col in df2.columns]

# Prepare a list to hold the mapping results
mapping_results = []
to_rename = {}
# Iterate over each column in df and find the best match in df2 using rapidFuzz
for col in df2_cols:
    best_match, score, _ = process.extractOne(col, df_cols, scorer=fuzz.QRatio)
    mapping_results.append({
        'df_column': col,
        'df2_column': best_match if score>50 else None,
        'similarity_score': score
    })
    if score>50:
        to_rename[best_match]=col

# Create a pandas DataFrame to display the mapping results including similarity score
mapping_df = pd.DataFrame(mapping_results)
mapping_df.head(22)

,df_column,df2_column,similarity_score
0,Month,Month,100.000000
1,Week,Week,100.000000
2,Year,None,25.000000
3,Level,Level,100.000000
4,Date,Date,100.000000
5,Week Ending Date,None,40.000000
6,Trips <1,Trips <1,94.117647
7,People Not Staying at Home,Population Not Staying at Home,85.714286
8,Population Staying at Home,Population Staying at Home,100.000000
9,Trips,Trips,100.000000


In [17]:
df['Level'].unique()

array(['National', 'State', 'County'], dtype=object)

In [18]:
df_national = df.groupby(by='Level').get_group('National').reset_index()
df_state = df.groupby(by='Level').get_group('State').reset_index()
df_county = df.groupby(by='Level').get_group('County').reset_index()
df_separate = [df_national,df_state,df_county]

In [19]:
[x.mean(numeric_only=True)['Number of Trips'] for x in df_separate]

[1184806140.6770256, 23231492.954451483, 435939.20349261153]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df_county_weekly = df_county.groupby('Week')['Number of Trips'].median()
df_state_weekly = df_state.groupby('Week')['Number of Trips'].sum()
df_national_weekly = df_national.groupby('Week')['Number of Trips'].sum()

# Plotting using seaborn/matplotlib
plt.figure(figsize=(12,6))
sns.lineplot(data=df_national_weekly, x='Week', y='Number of Trips', label='National', marker='o')
sns.lineplot(data=df_state_weekly, x='Week', y='Number of Trips', label='State', marker='s')
sns.lineplot(data=df_county_weekly, x='Week', y='Number of Trips', label='County (Median)', marker='^')

plt.xlabel("Week")
plt.ylabel("Number of Trips")
plt.title("Weekly Number of Trips by Aggregation Level")
plt.legend()
plt.tight_layout()
plt.show()


TypeError: Data source must be a DataFrame or Mapping, not <class 'pandas.core.series.Series'>.

<Figure size 1200x600 with 0 Axes>

In [ ]:

target_date = "6/12/2019"  

grouping_columns = ["State FIPS", "State Postal Code"]
aggregation_columns = [
    "Population Staying at Home", "Population Not Staying at Home", "Number of Trips",
    "Number of Trips <1", "Number of Trips 1-3", "Number of Trips 3-5",
    "Number of Trips 5-10", "Number of Trips 10-25", "Number of Trips 25-50",
    "Number of Trips 50-100", "Number of Trips 100-250", "Number of Trips 250-500",
    "Number of Trips >=500"
]

df_county_filtered = df_county[df_county["Date"] == target_date]
df_state_filtered = df_state[df_state["Date"] == target_date]

# Aggregate county-level data by state
county_agg = df_county_filtered.groupby(grouping_columns)[aggregation_columns].sum().reset_index()

# Merge with state-level data
merged_df = county_agg.merge(df_state_filtered, on=grouping_columns, suffixes=("_county", "_state"))


for col in aggregation_columns:
    merged_df[f"{col}_diff"] = merged_df[f"{col}_county"] - merged_df[f"{col}_state"]

# Select relevant columns for final dataset
comparison_df = merged_df[["State FIPS", "State Postal Code"] + [f"{col}_diff" for col in aggregation_columns]]

# Display the dataframe
print(comparison_df)


    State FIPS State Postal Code  Population Staying at Home_diff  \
0          1.0                AL                              0.0   
1          2.0                AK                          -1771.0   
2          4.0                AZ                              0.0   
3          5.0                AR                              0.0   
4          6.0                CA                           -219.0   
5          8.0                CO                              0.0   
6          9.0                CT                              0.0   
7         10.0                DE                              0.0   
8         11.0                DC                              0.0   
9         12.0                FL                              0.0   
10        13.0                GA                              0.0   
11        15.0                HI                            -19.0   
12        16.0                ID                              0.0   
13        17.0                IL  

In [ ]:
[abs((x/y)*100) for x,y in zip(comparison_df.mean(numeric_only=True),df_state.mean(numeric_only=True))]

[100.0,
 nan,
 2.8215481284645927,
 0.03357533847653602,
 0.0018543961957730283,
 0.008351237574351989,
 0.003748543994692996,
 0.008188991500593792,
 0.005161438278969556,
 0.0023915487364010316,
 0.0022830427145932447,
 0.0024030299751474863,
 0.0012039090479018886,
 0.008379067051361011]